# Compare results

## Setup

In [1]:
import pandas as pd
import numpy as np
import pickle 
from functools import reduce
import utils

RESULTS_FILENAME_LIST = [
    "./results/dtw_results_1658168584_00.pkl",
    "./results/hmm_results_1658168584_00.pkl"
    ]

## Load and merge dataframes with predictions

In [2]:
results_lst = []
for file in RESULTS_FILENAME_LIST:
    with open(file, "rb") as f:
        results_lst.append(pickle.load(f))

results_lst[0].keys()

dict_keys(['data_filename', 'time_index', 'split_id', 'results_df'])

In [3]:
df_lst = [r["results_df"] for r in results_lst]
merged_results_df = reduce(
    lambda df1, df2: pd.merge(
                              df1, df2, 
                              on = ["sample_id", "true_label"], 
                              how = "outer"), 
                    df_lst)

In [4]:
merged_results_df

,true_label,sample_id,predicted_label_1NN,predicted_label_5NN,predicted_label_AIC,predicted_label_BIC
0,0,1,0,0,0,0
1,0,2,0,0,0,0
2,0,3,0,0,0,0
3,0,4,0,0,0,0
4,0,5,0,0,0,0
...,...,...,...,...,...,...
355,8,891,8,8,8,8
356,8,893,8,8,8,8
357,8,895,8,8,8,8
358,8,896,8,8,8,8


## Show accuracy and train/test subsets size

In [5]:
true = merged_results_df["true_label"]
for preds_col in merged_results_df.iloc[:,2:]:
    print(f"{preds_col} accuracy: {utils.accuracy(true, merged_results_df[preds_col])}")

predicted_label_1NN accuracy: 0.9416666666666667
predicted_label_5NN accuracy: 0.9111111111111111
predicted_label_AIC accuracy: 0.9972222222222222
predicted_label_BIC accuracy: 0.9972222222222222


In [6]:
data_filename = results_lst[0]["data_filename"]
with open(data_filename, "rb") as f:
    data = pickle.load(f)
print(data["metadata"])

{'samples_per_model': 100, 'max_sample_len': 50, 'train_subset_size': 0.6, 'different_lenghts': False}


## Find wrong labels

In [21]:
wrong_samples = {}
for col in utils.find_colnames_with(merged_results_df):
    is_wrong = merged_results_df[col] != merged_results_df["true_label"]
    ids = np.where(is_wrong)[0]
    wrong_samples[col] = merged_results_df.iloc[ids]["sample_id"].values

for col, ids in wrong_samples.items():
    print(f"Ids of {len(ids)} misclassified samples in column {col}:")
    print(ids)

Ids of 21 misclassified samples in column predicted_label_1NN:
[ 33  54  66  85  94 307 308 328 336 339 348 349 351 359 364 380 385 388
 395 680 734]
Ids of 32 misclassified samples in column predicted_label_5NN:
[ 32  33  54  66  67  82  85 307 308 311 317 320 325 326 328 336 339 343
 348 349 351 359 364 380 385 386 388 395 396 606 680 737]
Ids of 1 misclassified samples in column predicted_label_AIC:
[63]
Ids of 1 misclassified samples in column predicted_label_BIC:
[63]


### Plots